In [1]:
from flask import Flask,request, render_template
from joblib import load
import pandas as pd
import numpy as np

In [2]:
model = load('bangalore_house_price_prediction_model2.pkl')

In [3]:
test_df = pd.read_csv('reduce_cat_class.csv')
test_df

,size,bath_imp,balcony_imp,price_imp,total_sqft_int,Carpet Area,Plot Area,Super built-up Area,availability_Ready To Move,location_Whitefield,...,societySoiewre,societyDLhtsan,societySoechHa,societyRInceeg,societyPrrtht,societyShthi S,societyEladero,societyBrway G,societyAsastDe,societyDLhtsen
0,2,2.0,1.000000,39.07,1056.0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,5.0,3.000000,120.00,2600.0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3,2.0,3.000000,62.00,1440.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,3,3.0,1.000000,95.00,1521.0,0,0,1,1,0,...,1,0,0,0,0,0,0,0,0,0
4,2,2.0,1.000000,51.00,1200.0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12856,5,4.0,0.000000,231.00,3453.0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
12857,4,5.0,1.584376,400.00,3600.0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
12858,2,2.0,1.000000,60.00,1141.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
12859,4,4.0,1.000000,488.00,4689.0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
scale.fit_transform(test_df.drop('price_imp',axis=1))

array([[-0.69674217, -0.56315911, -0.72988358, ..., -0.03946532,
        -0.03946532, -0.03946532],
       [ 1.33841591,  2.44233036,  1.78786714, ..., -0.03946532,
        -0.03946532, -0.03946532],
       [ 0.32083687, -0.56315911,  1.78786714, ..., -0.03946532,
        -0.03946532, -0.03946532],
       ...,
       [-0.69674217, -0.56315911, -0.72988358, ..., -0.03946532,
        -0.03946532, -0.03946532],
       [ 1.33841591,  1.44050054, -0.72988358, ..., -0.03946532,
        -0.03946532, -0.03946532],
       [-1.7143212 , -1.56498893, -0.72988358, ..., -0.03946532,
        -0.03946532, -0.03946532]])

In [5]:
def predict_house_price(model,size,bath_imp,balcony_imp,total_sqft_int,area_type,availability,location,society):
    

    x =np.zeros(len(test_df.columns)-1) # create zero numpy array, len = 107 as input value for model

  
    x[0]=size
    x[1]=bath_imp
    x[2]=balcony_imp
    x[3]=total_sqft_int
    

    if "availability"=="Ready To Move":
        x[8]=1

    if area_type in test_df.columns:
        area_type_index = np.where(test_df.columns==area_type)[0][0]
        x[area_type_index] =1

   
    if 'location_'+location in test_df.columns:
        loc_index = np.where(test_df.columns=="location_"+location)[0][0]
        x[loc_index] =1
        
   
        
    

   
    x = scale.transform([x])[0]

    return model.predict([x])[0] 

In [ ]:
app = Flask(__name__)


@app.route('/')
def home():
    return render_template('index.html')


@app.route('/',methods=['GET','POST'])
def predict():
    if request.method=='POST':
     
        input_features = request.form
        size = int(input_features['size'])
        bath_imp = int(input_features['bath_imp'])
        balcony_imp = int(input_features['balcony_imp'])
        total_sqft_int = int(input_features['total_sqft_int'])
        area_type = input_features['area_type']
        availability = input_features['availability']
        location = input_features['location']
        society = input_features['society']



        predicted_price = predict_house_price(model,size,bath_imp,balcony_imp,total_sqft_int,area_type,availability,location,society)


    return render_template('index.html',price=predicted_price)


    
if __name__ == "__main__":
    app.run()
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Jan/2021 20:08:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2021 20:09:33] "POST / HTTP/1.1" 200 -
